In [ ]:
!pip install PyPDF2
!pip install sentence-transformers
!pip install langchain
!pip install langchain_community
!pip install PyPDF2 sentence-transformers langchain faiss-cpu tiktoken llama-cpp-python
!pip install llama-cpp-python
!pip install tiktoken
!pip install gradio
!pip install pickle5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [ ]:
pwd


In [ ]:
import requests
import os

def download_model(url, destination_path):
    # Stream the file from the URL in chunks
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(destination_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# Example URL where the model file is hosted
model_url = "https://huggingface.co/TheBloke/stablelm-zephyr-3b-GGUF/resolve/main/stablelm-zephyr-3b.Q4_K_M.gguf?download=true"

# Destination path where the model file will be saved
model_path = "stablelm-zephyr-3b.Q4_K_M.gguf"

# Download the model file
download_model(model_url, model_path)




In [ ]:
import PyPDF2

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import gradio as gr
import pickle5 as pickle




In [ ]:

# pdf_docs=["Vinay Kumar Resume.pdf"]

In [ ]:

def prepare_docs(pdf):
    docs = []
    metadata = []
    content = []

    # for pdf in pdf_docs:

    pdf_reader = PyPDF2.PdfReader(pdf)
    for index, text in enumerate(pdf_reader.pages):
        doc_page = {'title': pdf + " page " + str(index + 1),
                    'content': pdf_reader.pages[index].extract_text()}
        docs.append(doc_page)
    for doc in docs:
        content.append(doc["content"])
        metadata.append({
            "title": doc["title"]
        })
    print("Content and metadata are extracted from the documents")
    return content, metadata



In [ ]:



def get_text_chunks(content, metadata):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,
        chunk_overlap=256,
    )
    split_docs = text_splitter.create_documents(content, metadatas=metadata)
    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs


In [ ]:
def ingest_into_vectordb(split_docs):
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    db = FAISS.from_documents(split_docs, embeddings)

    DB_FAISS_PATH = 'vectorstore/db_faiss'
    db.save_local(DB_FAISS_PATH)
    return db

In [ ]:
template = """[INST]
As an AI, provide accurate and relevant information based on the provided document. Your responses should adhere to the following guidelines:
- Answer the question based on the provided documents.
- Be direct and factual, limited to 50 words and 2-3 sentences. Begin your response without using introductory phrases like yes, no etc.
- Maintain an ethical and unbiased tone, avoiding harmful or offensive content.
- If the document does not contain relevant information, state "I cannot provide an answer based on the provided document."
- Avoid using confirmatory phrases like "Yes, you are correct" or any similar validation in your responses.
- Do not fabricate information or include questions in your responses.
- do not prompt to select answers. do not ask me questions
{question}
[/INST]
"""

#template = """Given the document and the current conversation between a user and an agent, your task is as follows: Answer any user query by using information from the document. The response should be detailed."""
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
def get_conversation_chain(vectordb):
    llama_llm = LlamaCpp(
    model_path="stablelm-zephyr-3b.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=200,
    top_p=1,
    callback_manager=callback_manager,
    n_ctx=3000)

    retriever = vectordb.as_retriever()
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(template)

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')

    conversation_chain = (ConversationalRetrievalChain.from_llm
                          (llm=llama_llm,
                           retriever=retriever,
                           #condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                           memory=memory,
                           return_source_documents=True))
    print("Conversational Chain created for the LLM using the vector store")
    return conversation_chain


In [ ]:
file = '/content/Student Allotment Letter harshal(1).pdf'
content, metadata = prepare_docs(file)
split_docs = get_text_chunks(content, metadata)
vectordb = ingest_into_vectordb(split_docs)

In [ ]:
conversation_chain=get_conversation_chain(vectordb)

In [ ]:
# !pip install pickle5

In [ ]:
with open('conversation_chain.pkl', 'wb') as f:
  pickle.dump(conversation_chain, f)


In [ ]:
with open('conversation_chain.pkl', 'rb') as f:
  conversation_chain = pickle.load(f)

In [ ]:
user_question = "what is the document about?"
response=conversation_chain({"question": user_question})
print("Q: ",user_question)
print("A: ",response['answer'])

In [ ]:

def greet(query,file):
  content, metadata = prepare_docs(file)
  split_docs = get_text_chunks(content, metadata)
  vectordb = ingest_into_vectordb(split_docs)
  conversation_chain=get_conversation_chain(vectordb)
  user_question = query
  response=conversation_chain({"question": user_question})
  print("Q: ",user_question)
  print("A: ",response['answer'])
  return response['answer']

    # vectorstore = generate_embeddings(file)
    # ##retriever    # retriever = vectorstore.as_retriever(
    # search_type="mmr", #similarity
    # search_kwargs={'k': 4})

    # llm = HuggingFaceHub(
    # repo_id="huggingfaceh4/zephyr-7b-alpha",
    # model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512},cache_dir = r"C:\Users\janak.joshi\Desktop\gradio\flagged\model")

    # query = name

    # prompt = f"""
    # <|system|>
    # You are an AI assistant that follows instruction extremely well.
    # Please be truthful and give direct answers.
    # also don't go outside the context, if there is something outside the context just say you don't know.
    # </s>
    # <|user|>
    # {query}
    # </s>
    # <|assistant|>
    # """

    # qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever)

    # response = qa.run(prompt)



    # # print(page)

    # return "Hello " + name + "!!" +"and the answer is -- " + " " +response

iface = gr.Interface(greet, ["text","file"], "text")
iface.launch(share=True,debug=True)


In [ ]:
# Below code For Advance Experiment

In [ ]:
!pip install streamlit

In [ ]:

import streamlit as st

# Define global variables to store processed document and vector database
vectordb = None

def process_document_once(file):
    # Function to process the uploaded document and create a vector database only once
    global vectordb
    if vectordb is None:
        content, metadata = prepare_docs(file)
        split_docs = get_text_chunks(content, metadata)
        vectordb = ingest_into_vectordb(split_docs)

def get_answer(query):
    # Function to retrieve answer from the vector database based on user's query
    conversation_chain = get_conversation_chain(vectordb)
    response = conversation_chain({"question": query})
    return response['answer']

def main():
    st.title("Document Query Answering System")

    page = st.sidebar.radio("Navigation", ["Upload Document", "Ask Questions"])

    if page == "Upload Document":
        st.header("Step 1: Upload Document")
        uploaded_file = st.file_uploader("Upload document", type=["txt", "pdf"])
        if uploaded_file is not None:
            process_document_once(uploaded_file)
            st.success("Document uploaded successfully! Now you can proceed to ask questions.")
    elif page == "Ask Questions":
        st.header("Step 2: Ask Questions")
        if vectordb is None:
            st.warning("Please upload a document first.")
        else:
            query = st.text_input("Enter your question:")
            if st.button("Get Answer"):
                answer = get_answer(query)
                st.write(f"For the question '{query}', the answer is: {answer}")

if __name__ == "__main__":
    main()


In [ ]:

!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py

In [ ]:

#####3 final iteration

In [ ]:
# Iteration 2



def greet(query,file):
  content, metadata = prepare_docs(file)
  split_docs = get_text_chunks(content, metadata)
  vectordb = ingest_into_vectordb(split_docs)
  conversation_chain=get_conversation_chain(vectordb)
  user_question = query
  response=conversation_chain({"question": user_question})
  print("Q: ",user_question)
  print("A: ",response['answer'])
  return "Hello, for question  " + query + "!!" +"and the answer is -- " + " " +response['answer']

    # vectorstore = generate_embeddings(file)
    # ##retriever
    # retriever = vectorstore.as_retriever(
    # search_type="mmr", #similarity
    # search_kwargs={'k': 4})

    # llm = HuggingFaceHub(
    # repo_id="huggingfaceh4/zephyr-7b-alpha",
    # model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512},cache_dir = r"C:\Users\janak.joshi\Desktop\gradio\flagged\model")

    # query = name

    # prompt = f"""
    # <|system|>
    # You are an AI assistant that follows instruction extremely well.
    # Please be truthful and give direct answers.
    # also don't go outside the context, if there is something outside the context just say you don't know
    # </s>
    # <|user|>
    # {query}
    # </s>
    # <|assistant|>
    # """

    # qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever)
    # response = qa.run(prompt)
    # # print(page)

    # return "Hello " + name + "!!" +"and the answer is -- " + " " +response

iface = gr.Interface(greet, ["text","file"], "text")
iface.launch(share=True)


In [ ]:
pwd

In [ ]:
# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("/content/djangoapp.zip", 'r') as zObject:

	# Extracting all the members of the zip
	# into a specific location.
	zObject.extractall(
		path="/content")


In [ ]:
import threading

def run_server():
    !python main.py runserver

# Start the Django development server in a separate thread
server_thread = threading.Thread(target=run_server)
server_thread.start()

In [ ]:
from pyngrok import ngrok

# Function to setup ngrok tunnel
def setup_ngrok():
    ngrok.set_auth_token("2fOYbRXkWLrmz48Gu1gWqM7BgTC_2YR85vXtW65Zy1WP6ZE7J")
    public_url = ngrok.connect(addr="8000", proto="http")
    print("Ngrok Tunnel URL:", public_url)

# Setup ngrok tunnel in a separate thread
ngrok_thread = threading.Thread(target=setup_ngrok)
ngrok_thread.start()

In [ ]:
!python manage.py runserver

In [ ]:
pwd

In [ ]:
!python manage.py runserver

In [ ]:
!ngrok http 8000

In [ ]:
!ngrok config add-authtoken 2T6YXZSWoBe4TxXRUNrEcqUorwa_5SCckqKM6EwuaTsiUXbyc